In [ ]:
#htsf\Scripts\activate    

# Import libraries
import pandas as pd
import numpy as np
from plotnine import ggplot, aes, geom_line

import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'src'))
import hyperarch

%matplotlib inline


def get_data(file_loc, verbose=True):
    data = pd.read_csv(file_loc)
    # Update Data Types
    data['date'] = pd.to_datetime(data['date'])
    data['subcategory'] = data['subcategory'].astype('category')
    data['category'] = data['category'].astype('category')
    if verbose is True:
        display(data.head())
        data.info()
        for col in data.columns:
            if data[col].dtype == 'category':
                print(f'{col} unique values: {len(data[col].unique())}')
            elif data[col].dtype == 'datetime64[ns]':
                print(f'Min Date: {data[col].min()}')
                print(f'Max Date: {data[col].max()}')
            else:
                print(f'{col}:\n{data[col].describe()}\n')
    return data

# Read data
h_df = get_data('../data/hierarchal_agg.csv', verbose = True)
g_df = get_data('../data/grouped_agg.csv', verbose = False)

In [ ]:
# Hierarchy
hierarchy_df, h_bottom, h_labels = hyperarch.get_hierarchal(h_df, 'category', 'subcategory', agg_type='hierarchy')   
h_s = hyperarch.get_S(h_bottom, h_labels, agg_type='hierarchy')
h_models = hyperarch.get_models(hierarchy_df, order=(1,1,0), steps_out=1)
h_yhat = hyperarch.get_forecast_matrix(h_models)
h_reconciled_yhat = hyperarch.reconcile(h_yhat, h_s, method='ols')
hdf_pred, hdf_rec = hyperarch.predict_hier(hierarchy_df, h_yhat, h_reconciled_yhat, h_labels, h_models['index'])

# Grouped
grouped_df, g_bottom, g_labels = hyperarch.get_hierarchal(g_df, 'category', 'subcategory', agg_type='grouped')   
g_s = hyperarch.get_S(g_bottom, g_labels, agg_type='grouped')
g_models = hyperarch.get_models(grouped_df, steps_out=1)
g_yhat = hyperarch.get_forecast_matrix(g_models)
g_reconciled_yhat = hyperarch.reconcile(g_yhat, g_s, method='ols')
gdf_pred, gdf_rec = hyperarch.predict_hier(grouped_df, g_yhat, g_reconciled_yhat, g_labels, g_models['index'])

In [ ]:
import utils

h_df.groupby(['category', 'subcategory']).agg(append_years)

In [ ]:
(
    ggplot(hdf_pred)
    + aes(x=hdf_pred.index, y='total', color='actual')
    + geom_line()
)